In [1]:
import pandas as pd
import unicodedata
import numpy as np
from geopy.geocoders import Nominatim
from geopy.geocoders import Bing
from dotenv import load_dotenv
import os

load_dotenv()

# df = pd.read_csv('./input/main_lodz_full20230802-143112.csv',delimiter=';')
df = pd.read_csv('./input/main_wawa_full20230808-171401.csv',delimiter=';',encoding='utf-8',encoding_errors='replace')


In [2]:

df.head()

,date,link,name,where,price,perm,rooms,sqm,who
0,20230808,/pl/oferta/dwa-pokoje-na-nowolipkach-ID4kzNA,Dwa pokoje na Nowolipkach,"ul. Nowolipki 25B, Nowolipki, Wola, Warszawa, ...",610 000 zł,16 176 zł/m²,2 pokoje,37.71 m²,Oferta prywatna
1,20230808,/pl/oferta/mieszkanie-warszawa-wilanow-ID4mCoz,Mieszkanie - Warszawa Wilanów,"al. Aleja Rzeczypospolitej, Błonia Wilanowskie...",1 250 000 zł,17 007 zł/m²,3 pokoje,73.5 m²,Eurovilla Miasteczko Wilanów Sp. z o. o.Biuro ...
2,20230808,/pl/oferta/seg-srodkowy-strych-dzialka-taras-b...,Seg. środkowy+strych+działka+ taras+ balkon+garaż,"ul. Natalii Gąsiorowskiej, Nowodwory, Białołęk...",950 000 zł,6565 zł/m²,5 pokoi,144.7 m²,DSM DEWELOPMENTInwestycja deweloperska
3,20230808,/pl/oferta/doskonala-lokalizacja-41-m-w-srodmi...,"Doskonala lokalizacja! 41 m² w Śródmieściu, Złota","ul. Złota, Śródmieście Północne, Śródmieście, ...",735 000 zł,17 927 zł/m²,2 pokoje,41 m²,HOMEBiuro nieruchomości
4,20230808,/pl/oferta/mieszkanie-na-osiedlu-jutrzenki-114...,Mieszkanie na osiedlu Jutrzenki - 114/M030,"ul. Jutrzenki, Salomea, Włochy, Warszawa, mazo...",663 000 zł,14 668 zł/m²,2 pokoje,45.2 m²,Oferta prywatna


In [3]:
def after_dot(string:str):
    return string[string.find('.')+1:]
    
def type_clasificator(who:str):
    who = who.lower()
    if 'oferta prywatna' in who:
        return 'P'
    if 'inwestycja deweloperska' in who:
        return 'D'
    if 'biuro nieruchomości' in who:
        return 'N'
    return 'X'

def history_fill(main_df:pd.DataFrame,source_df:pd.DataFrame,fill_col:str,join_col:str):
    main_df[fill_col].fillna(pd.merge(main_df,source_df,how='left',on=join_col,suffixes=('', '_new'))[fill_col+'_new'],inplace=True)

def add_empty_col(df:pd.DataFrame,colname:str):
    if colname not in df.columns:
        df[colname] = np.nan
        
def geolocate_empty(df):
    # before geocoding
    ## prepare known list
    without_dupli = df.drop_duplicates('where')
    known_without_dupli = without_dupli[without_dupli['latitude'].notna()]
    ## first geocode base on already known
    for col in ['latitude','longitude','neighbourhood','suburb','postcode']:
        history_fill(df,known_without_dupli,col,'where')

    new_without_dupli = without_dupli.loc[df['latitude'].isna()]['where'].to_frame()
    print(f'{new_without_dupli.shape[0]} - records needs geocoding')
    ## only new without gps remains, need geocoding
    ## GEOCODING
    new_without_dupli_gps = new_without_dupli.apply(add_geo,axis=1)
    ## now fill based on new data
    for col in ['latitude','longitude','neighbourhood','suburb','postcode']:
        history_fill(df,new_without_dupli_gps,col,'where')

In [4]:
# drop error lines
display(df.shape)
df=df[df['date'].str.isdigit()]
df.shape


(11342, 9)

(11329, 9)

In [5]:
# add missing columns
for col in ['latitude','longitude','neighbourhood','suburb','postcode']:
        add_empty_col(df,col)
# unicode normalization
df['perm'] = df['perm'].fillna('brak').apply(lambda x: unicodedata.normalize('NFKC',x)).replace('brak',np.nan)
df['price'] = df['price'].fillna('brak').apply(lambda x: unicodedata.normalize('NFKC',x)).replace('brak',np.nan).str.replace(',','.')


In [6]:
# enrich fields
df['link'] = 'http://www.otodom.pl' + df['link']
df['sqm'] = df['sqm'].str.replace(' m²','').astype(float)
df['perm'] = df['perm'].str.replace('zł/m2','').str.replace('€/m2','').str.replace(' ','').str.replace(',','.').fillna(-1).astype(float).astype(int)
df['price'] = df['price'].str.replace('zł','').str.replace('€','').str.replace('Zapytaj o cenę','-1').str.replace(' ','').str.replace(',','.').fillna(-1).astype(float).astype(int)
df['rooms'] = df['rooms'].str.replace(r' pokoje| pokoi| pokój','',regex=True).str.replace('+','').astype(int)
# add new
df['id'] = df['link'].str[-7:]
df['type'] = df['who'].apply(type_clasificator)
# type adjustment
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
# clean
df.replace({'who' : { 'Inwestycja deweloperska' : '', 'Biuro nieruchomości' : '' }}, inplace=True,regex=True)

C:\Users\krawc\AppData\Local\Temp\ipykernel_15896\893869728.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rooms'] = df['rooms'].str.replace(r' pokoje| pokoi| pokój','',regex=True).str.replace('+','').astype(int)


In [7]:
#Geolocation 45min-5k records

def geo_bing(adress:str,short=False):
    bing = Bing(api_key=os.environ.get("BING_MAPS"))
    adress = adress.split(', ')
    if not short:
        query_dict = {'addressLine':adress[0],
            'locality':adress[-2],
            'adminDistrict':adress[-1],
            'countryRegion':'Poland'}
        # print('qb',query_dict)
    else:
        query_dict = {'addressLine':adress[-3],
            'locality':adress[-2],
            'adminDistrict':adress[-1],
            'countryRegion':'Poland'}

    data = bing.geocode(query_dict, exactly_one=True, include_neighborhood=True)
    rawdict = data.raw['address']

    return (data.latitude,data.longitude,rawdict)

def geo_nominatim(adress:str,county=True):
    
    nominatim = Nominatim(user_agent="http")

    adress = adress.split(', ')
    if county:
        query_dict = {'street':after_dot(adress[0]),
            'city':adress[-2],
            'county':adress[-3],
            'state':adress[-1],
            'country':'pl'}
    else:
        query_dict = {'street':after_dot(adress[0]),
            'city':adress[-2],
            'state':adress[-1],
            'country':'pl'}
        
    data = nominatim.geocode(query_dict,country_codes='pl',exactly_one=True,addressdetails=True)
    try:
        ndict = data.raw['address']
        return (data.latitude,data.longitude,ndict)
    except:
        return
    
def geo_nominatim_postal(postcode:str):
    
    nominatim = Nominatim(user_agent="http")
    query_dict = {'postalcode':postcode}     
    data = nominatim.geocode(query_dict,country_codes='pl',exactly_one=True,addressdetails=True)
    ndict = data.raw['address']
    return (data.latitude,data.longitude,ndict)

def geo_nominatim_gps(la,lo):
    print('gps')
    nominatim = Nominatim(user_agent="http")
    query = (la,lo)     
    data = nominatim.reverse(query,exactly_one=True,addressdetails=True)
    ndict = data.raw['address']
    return (data.latitude,data.longitude,ndict)
    
def get_geo(adress):
    
    # try with county
    geo = geo_nominatim(adress,county=True)
    if geo:
        return geo
    # try without county    
    geo = geo_nominatim(adress,county=False)
    if geo:
        return geo
    # try with bing and return to nominatim
    geo = geo_bing(adress)
    if geo:
        la,lo,d=geo
        # print(1,d)

        if 'postalCode' in d.keys():
            try:
                geo = geo_nominatim_postal(d['postalCode'])
            except:
                try:
                    geo = geo_nominatim_gps(la,lo)
                except:
                    pass
        else:
            try:
                geo = geo_bing(adress,short=True)
                _,_,d=geo
                # print(2,d)
                geo = geo_nominatim_postal(d['postalCode'])
            except:
                return
        return geo

def add_geo(series:pd.Series):
    
    try:
        la,lo,d = get_geo(series['where'])
        if 'neighbourhood' not in d.keys():
            if 'quarter' in d.keys():
                d['neighbourhood'] = d['quarter']
            else:
                d['neighbourhood'] = 'brak'
        if 'suburb' not in d.keys():
            if 'city_district' in d.keys():
                d['suburb'] = d['city_district']
            else:
                d['suburb'] = 'brak'
        if 'postcode' not in d.keys():
            d['postcode'] = d['postalCode']

        series['latitude'],series['longitude'],series['neighbourhood'],series['suburb'],series['postcode'] = la,lo,d['neighbourhood'],d['suburb'],d['postcode']
    except:
        print(series['where'])
    return series

In [8]:
geolocate_empty(df)
# 2029-20minut

C:\Users\krawc\AppData\Local\Temp\ipykernel_15896\3397996781.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  known_without_dupli = without_dupli[df['latitude'].notna()]


2029 - records needs geocoding
Śródmieście Południowe, Śródmieście, Warszawa, mazowieckie
Śródmieście Północne, Śródmieście, Warszawa, mazowieckie
okolice Park Łazienek, Ujazdów, Śródmieście, Warszawa, mazowieckie
Stary Mokotów, Mokotów, Warszawa, mazowieckie
Ulrychów, Wola, Warszawa, mazowieckie
Pyry, Ursynów, Warszawa, mazowieckie
Huta, Bielany, Warszawa, mazowieckie
Błonia Wilanowskie, Wilanów, Warszawa, mazowieckie
Groszówka, Wesoła, Warszawa, mazowieckie
Gocławek, Praga-Południe, Warszawa, mazowieckie
Piaski, Bielany, Warszawa, mazowieckie
Szczęśliwice, Ochota, Warszawa, mazowieckie
Czerniaków, Mokotów, Warszawa, mazowieckie
Stacja Wola, ul. Gumińska, Odolany, Wola, Warszawa, mazowieckie
Stare Włochy, Włochy, Warszawa, mazowieckie
Nowa inwestycja na pograniczu Bemowa i Woli. Blisko metra., Górce, Bemowo, Warszawa, mazowieckie
Jelonki Północne, Bemowo, Warszawa, mazowieckie
Zerzeń, Wawer, Warszawa, mazowieckie
Targówek Mieszkaniowy, Targówek, Warszawa, mazowieckie
Bemowo-Lotnisko, 

In [8]:
x= 'ul. Juliusza Ordona, Ulrychów, Wola, Warszawa, mazowieckie'
get_geo(x)
# geo_bing(x)


{'addressLine': 'ul. Juliusza Ordona', 'locality': 'Warszawa', 'adminDistrict': 'mazowieckie', 'countryRegion': 'Poland'}
1 {'addressLine': 'Juliusza Słowackiego', 'adminDistrict': 'Mazowieckie', 'adminDistrict2': 'Warsaw', 'countryRegion': 'Poland', 'formattedAddress': 'Juliusza Słowackiego, 01-690 Warsaw, Poland', 'locality': 'Warsaw', 'neighborhood': 'Żoliborz', 'postalCode': '01-690'}


(52.27238685,
 20.97013285,
 {'quarter': 'Marymont-Potok',
  'suburb': 'Żoliborz',
  'city': 'Warszawa',
  'state': 'województwo mazowieckie',
  'ISO3166-2-lvl4': 'PL-14',
  'postcode': '01-690',
  'country': 'Polska',
  'country_code': 'pl'})

In [9]:

df.sample(5)

,date,link,name,where,price,perm,rooms,sqm,who,latitude,longitude,neighbourhood,suburb,postcode,id,type
384,2023-08-08,http://www.otodom.pl/pl/oferta/rezydencja-liws...,Rezydencja Liwska | atrakcyjne mieszkanie M059,"ul. Liwska, Bródno, Targówek, Warszawa, mazowi...",-1,-1,2,41.29,LW Development,52.290051,21.017088,Toruńska,Targówek,03-391,ID4e4R9,D
459,2023-08-08,http://www.otodom.pl/pl/oferta/z-ogodkiem-blis...,"Z ogódkiem, blisko Pałacu Stary Wilanów","ul. Obornicka, Wilanów Królewski, Wilanów, War...",3800000,20392,5,186.35,Eurovilla Wilanów Sp. z o.o.,52.170400,21.086009,Stary Wilanów,Wilanów,02-946,ID4meRz,N
1398,2023-08-08,http://www.otodom.pl/pl/oferta/4-pokojowe-mies...,4-pokojowe mieszkanie 82m2 + balkon Bez Prowizji,"ul. Sławińska, Czyste, Wola, Warszawa, mazowie...",1436000,17474,4,82.18,Skanska,52.319869,21.078010,Grodzisk,Białołęka,00-999,ID4lgFd,D
758,2023-08-08,http://www.otodom.pl/pl/oferta/2-pokojowe-mies...,2-pokojowe mieszkanie 40m2 + balkon Bez Prowizji,"ul. Milewska, Pelcowizna, Praga-Północ, Warsza...",-1,-1,2,40.13,GH Development,52.253459,20.898094,Fort Radiowo,Bemowo,01-476,ID4mATK,D
858,2023-08-08,http://www.otodom.pl/pl/oferta/2-pokojowe-mies...,2-pokojowe mieszkanie 39m2 + balkon Bezpośrednio,"ul. Pawła Włodkowica, Brzeziny, Białołęka, War...",-1,-1,2,39.07,Victoria Dom,52.309160,21.025184,Białołęka Szlachecka,Białołęka,03-235,ID4mARN,D


In [10]:
# Export
df.to_csv('./out2.csv',sep=';',index=False)

In [ ]:
expert list dla brakujacych adresow
